# Information Retrieval using word2vec based Vector Space Model

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import pandas as pd
import re
import sys
sys.path.insert(0, '/home/guinzburg/NLP/Data')
from data_parser import parse_queries, parse_documents
from sklearn.model_selection import train_test_split
import spacy ## for stopwords removal
nlp = spacy.load('en_core_web_sm',disable=['ner','parser'])
nlp.max_length=5000000
from gensim.models import Word2Vec # for word2vec model
from sklearn.metrics.pairwise import cosine_similarity

Load the files containing queries and documents using 'parse_queries' and 'parse_documents'

In [2]:
# Get all queries
queries = pd.DataFrame.from_dict(parse_queries(path='/home/guinzburg/NLP/Data/CISI.QRY'), columns = ['query'], orient="index").reset_index(drop=True)
queries.index.name = 'id'

# Get all documents
documents = pd.DataFrame.from_dict(parse_documents(path='/home/guinzburg/NLP/Data/CISI.ALL'), orient="index").reset_index(drop=True)
documents.index.name = 'id'

Create a train set and test set for training the word2vec model for both queries and documents

In [3]:
# Create train and test set for queries (80% / 20%)
training_queries, testing_queries = train_test_split(queries, test_size=0.2)

# Create train and test set for documents (80% / 20%)
training_documents, testing_documents = train_test_split(documents, test_size=0.2)

# Text Preprocessing
Process the documents and queries as follows:

### Documents:
1. Lowercase the text
2. Expand Contractions
3. Clean the text
4. Remove Stopwords
5. Lemmatize words

### Queries:
1. Lowercase the text
2. Expand Contractions
3. Clean the text
4. We now have everything clear in our minds, so let’s start writing codes for pre-processing the documents.

In [4]:
# Create empty corpuses dataframes for training and testing
training_corpus = pd.DataFrame(columns=['cleaned','lemmatized', 'vector'])
testing_corpus = pd.DataFrame(columns=['cleaned','lemmatized', 'vector'])

# Lowercasing the text
training_corpus['cleaned'] = training_documents['body'].apply(lambda x:x.lower())
testing_corpus['cleaned'] = testing_documents['body'].apply(lambda x:x.lower())


def expand_contractions(text):
    """
        Description: function for expanding contractions in a given text
    """
    
    # Dictionary of english Contractions
    contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
    "'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
    "didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
    "hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
    "he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
    "i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
    "isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
    "let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
    "mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
    "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
    "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
    "she'll've": "she will have","should've": "should have","shouldn't": "should not",
    "shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
    "there'd": "there would","there'd've": "there would have",
    "they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
    "they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
    "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
    "weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
    "what've": "what have","when've": "when have","where'd": "where did",
    "where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
    "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
    "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
    "you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
    "you're": "you are","you've": "you have"}

    # Regular expression for finding contractions
    contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

    def replace(match):
        return contractions_dict[match.group(0)]
    
    return contractions_re.sub(replace, text)

def clean_text(text):
    """
        Description: function for cleaning a given text, 
        such as: 
        [*] remove the words with digits
        [*] replace newline characters with space
        [*] remove URLs
        [*] replace everything that isn’t English alphabets with space
    """
    
    text = re.sub('\w*\d\w*','', text)
    text = re.sub('\n',' ',text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub('[^a-z]',' ',text)
    
    return text

# Expanding Contractions
training_corpus['cleaned'] = training_corpus['cleaned'].apply(lambda x:expand_contractions(x))
testing_corpus['cleaned'] = testing_corpus['cleaned'].apply(lambda x:expand_contractions(x))

# Cleaning corpus using RegEx
training_corpus['cleaned'] = training_corpus['cleaned'].apply(lambda x: clean_text(x))
testing_corpus['cleaned'] = testing_corpus['cleaned'].apply(lambda x: clean_text(x))

# Removing extra spaces
training_corpus['cleaned'] = training_corpus['cleaned'].apply(lambda x: re.sub(' +',' ',x))
testing_corpus['cleaned'] = testing_corpus['cleaned'].apply(lambda x: re.sub(' +',' ',x))

# Removing Stopwords and Lemmatizing words
training_corpus['lemmatized'] = training_corpus['cleaned'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
testing_corpus['lemmatized'] = testing_corpus['cleaned'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

Rre-process queries.

In [5]:
# Create empty dataframes for queries training and testing
training_queries_ = pd.DataFrame(columns=['cleaned','lemmatized', 'vector'])
testing_queries_ = pd.DataFrame(columns=['cleaned','lemmatized', 'vector'])

# Lowercasing the text
training_queries_['cleaned'] = training_queries['query'].apply(lambda x:x.lower())
testing_queries_['cleaned'] = testing_queries['query'].apply(lambda x:x.lower())

# Expanding contractions
training_queries_['cleaned'] = training_queries_['cleaned'].apply(lambda x:expand_contractions(x))
testing_queries_['cleaned'] = testing_queries_['cleaned'].apply(lambda x:expand_contractions(x))

# Cleaning queries using RegEx
training_queries_['cleaned'] = training_queries_['cleaned'].apply(lambda x: clean_text(x))
testing_queries_['cleaned'] = testing_queries_['cleaned'].apply(lambda x: clean_text(x))

# Removing extra spaces
training_queries_['cleaned'] = training_queries_['cleaned'].apply(lambda x: re.sub(' +',' ',x))
testing_queries_['cleaned'] = testing_queries_['cleaned'].apply(lambda x: re.sub(' +',' ',x))

# Creating Vectors
Train the word2vec model and generate vectors for documents and queries in the testing set for information retrieval. First step is to prepare the dataset for training the model, and to use the same w2v model for generating vectors for both documents and queries,  we’ll combine both documents and queries training to create a single file

In [6]:
# Combining corpus and queries for training
combined_training = pd.concat([training_corpus.rename(columns={'lemmatized':'text'})['text'],\
                                 training_queries_.rename(columns={'cleaned':'text'})['text']]).sample(frac=1).reset_index(drop=True)

# Model Training
Using Gensim to train the word2vec model. Gensim is a python package used for topic modeling, text processing, and working with word vector models such as Word2Vec and FastText

In [11]:
# Creating data for the model training, iterate on each row from the combined data and split it in to words
train_data = []
for row in combined_training:
    train_data.append(row.split())

# Training a word2vec model from the given data set
# w2v_model = Word2Vec(train_data, vector_size=300, min_count=2, window=5, sg=1, workers=4)
# vector_size: default value 100 [https://stackoverflow.com/questions/45444964/python-what-is-the-size-parameter-in-gensim-word2vec-model-class]
# min_count: Words below the min_count frequency are dropped before training occurs. 
# window: [https://stackoverflow.com/questions/22272370/word2vec-effect-of-window-size-used/30447723#30447723]
w2v_model = Word2Vec(train_data, min_count=2)
print('Vocabulary size:', len(w2v_model.wv.index_to_key))
print('Model vector size:', w2v_model.wv.vector_size)

Vocabulary size: 4068
Model vector size: 100


In [12]:
def get_embedding_w2v(doc_tokens):
    """
    Description: function for generating vectors for the whole document or query. 
    This function will use the word2vec model and generate the vectors for each word in the document.
    Then, it will take the average of the vectors, and the resulting vector will represent the vector for the document. 
    Any document or query of length zero will have a vector containing zeroes, and any word which won’t be present in the vocabulary will have a vector with random values.
    """
    
    embeddings = []
    
    if len(doc_tokens) < 1:
        # w2v_model.wv.vector_size: 100 (default)
        return np.zeros(w2v_model.wv.vector_size)
    else:
        for tok in doc_tokens:
            if tok in list(w2v_model.wv.index_to_key):
                embeddings.append(w2v_model.wv.get_vector(tok))
            else:
                # w2v_model.wv.vector_size: 100 (default)
                embeddings.append(np.random.rand(w2v_model.wv.vector_size))
    
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector'] = testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x.split()))
testing_queries_['vector'] = testing_queries_['cleaned'].apply(lambda x :get_embedding_w2v(x.split()))

# Final IR Pipeline

In [19]:
def ranking_ir(query):
    """
        Description: a function that takes a query as an input and return the top 10 relevant documents. 
        This function will follow the information retrieval(IR) pipeline. First, it will pre-process the query. Then, it will generate the vector for it. After that, it will rank the documents based on the similarity scores.
    """      
    # pre-process Query
    query = re.sub(' +', ' ', clean_text(expand_contractions(query.lower())))

    # generating vector
    vector = get_embedding_w2v(query.split())

    # ranking documents
    documents_ = documents[['title','body']].copy()
    documents['similarity'] = testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
    documents.sort_values(by='similarity', ascending=False,inplace=True)

    return documents.head(10).reset_index(drop=True)

# Testing

In [20]:
ranking_ir("""How can actually pertinent data, as opposed to references or entire articles
themselves, be retrieved automatically in response to information requests?""")

,title,body,similarity
0,Data Retrieval Systems: Specifics and Problems,The essential differences between data retriev...,0.973983
1,Project Intrex: A General Review,A comprehensive review of the experimental inf...,0.971820
2,A Test for the Separation of Relevant and Non-...,Many retrieval experiments are intended to dis...,0.971063
3,Vocabulary Building and Control Techniques,The rationale is given for creation and mainta...,0.970813
4,Representation of Concept Relations Using the ...,Successful information retrieval from a mechan...,0.970574
5,The CA Integrated Subject File. II. Evaluation...,The relative retrieval performances of the CA ...,0.970516
6,"The identification of variable-length, equifre...",The words of natural language texts exhibit a ...,0.969469
7,A Business Intelligence System,An automatic system is being developed to diss...,0.969383
8,An Experiment in Index Term Frequency,This paper presents an experimental study of i...,0.969079
9,Minimum Vocabularies in Information Indexing,"Words have no precision, though in information...",0.969029


# Ranking & Evaluation

We have successfully trained our word2vec model and created vectors for documents and queries in the testing set for information retrieval. Now, it’s time to rank the documents according to the queries. But before that, we need to learn about the evaluation metric. Because only implementing a method won’t solve our purpose. We need a metric for checking the performance of our model.

The evaluation metric which we’ll be using here is Mean Average Precision(MAP@K)
Mean Average Precision is the mean of the average precision for all the queries used during evaluation. So, precision is calculated at each rank, average precision is calculated for a query, and mean average precision is calculated for the whole IR model. Now you know what MAP@K is. So, let’s write some code and calculate it for our vector space model.

For the ranking and evaluation, I have created a function average_precision(), which takes the query id and vector of a query as an input and returns the average precision value.
The value of MAP ranges between 0 and 1, with zero being the worst and one as best

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Function for calculating average precision for a query
def average_precision(qid,qvector):
  
  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')
  
  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values
  
  # Calculating precision
  precision=[]
  for i in range(1,11):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)
  
  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())